# Abstract Factory

> If you have a hierarchy of types, then you can have a corresponding hierarchy of factories.

Let's implement a `HotDrink` abstract base class and implement a few hot drinks along.

In [1]:
from abc import ABC, abstractmethod

class HotDrink(ABC):
    @abstractmethod
    def consume(self):
        pass

class Tea(HotDrink):
    def consume(self):
        print('This tea is nice but I\'d prefer it with milk')

class Coffee(HotDrink):
    def consume(self):
        print('This coffee is delicious')

Now let's suppose that preparating any of these drinks is a process sophisticated enough that they merit factories. Just like with `HotDrink`, we can create an abstract base factory class and implement separate factories from it, one for each hot drink implementation:

In [2]:
class HotDrinkFactory(ABC):
    @abstractmethod
    def prepare(self, amount):
        pass

class TeaFactory(HotDrinkFactory):
    def prepare(self, amount):
        print(f'Put in tea bag, boil water, pour {amount}ml, enjoy!')
        return Tea()


class CoffeeFactory(HotDrinkFactory):
    def prepare(self, amount):
        print(f'Grind some beans, boil water, pour {amount}ml, enjoy!')
        return Coffee()

So, how do we make a drink? Here's a simple scenario:

In [3]:
# If you're running this cell in VS Code, you'll get an input prompt at the top of the window when you run this cell

def make_drink(type):
    if type == 'tea':
        return TeaFactory().prepare(200)
    elif type == 'coffee':
        return CoffeeFactory().prepare(50)
    else:
        return None

entry = input('What kind of drink would you like?')
drink = make_drink(entry)
drink.consume()

Grind some beans, boil water, pour 50ml, enjoy!
This coffee is delicious


As it currently stands, the only thing that the `HotDrinkFactory` is useful for is to mandate an API for our factories.

We can organize things a little bit better by creating a separate component to make use of the different factories and stick them into a collection of sorts. Let's call this component `HotDrinkMachine`:

In [4]:
from enum import Enum, auto

class HotDrinkMachine:
    class AvailableDrink(Enum):  # violates OCP
        COFFEE = auto()
        TEA = auto()
    
    factories = [] # We store our factory instances here
    initialized = False # For making sure that we initialized the factories

    def __init__(self):
        if not self.initialized:
            self.initialized = True
            for d in self.AvailableDrink:
                name = d.name[0] + d.name[1:].lower() # we convert the enum "COFFEE" to just "ºCoffee"º
                factory_name = name + 'Factory'
                factory_instance = eval(factory_name)() # we instantiate the factory...
                self.factories.append((name, factory_instance)) # ...and add it to our factories list as a tuple.

    # Let's publicize our factories
    def make_drink(self):
        print('Available drinks:')
        for idf, f in enumerate(self.factories):
            print(f'{idf} - {f[0]}')

        s = input(f'Please pick drink (0-{len(self.factories)-1}): ')
        idx = int(s)
        s = input(f'Specify amount: ')
        amount = int(s)
        return self.factories[idx][1].prepare(amount) # the [1] is for accessing the factory instance within the tuple

> Note that we would have to modify our enum evey time we add a new drink, which would violate the Open-Closed Principle.

We have made our factories available in our new `HotDrinkMachine` class. Let's see it in action:

In [5]:
hdm = HotDrinkMachine()
drink = hdm.make_drink()
drink.consume()

Available drinks:
0 - Coffee
1 - Tea
Put in tea bag, boil water, pour 250ml, enjoy!
This tea is nice but I'd prefer it with milk


Final note: besides the API mandate, the abstract base factory class doesn't really do anything and we could delete it and refactor our factories to not make use of it, and the code would still continue to work, because Python uses [Duck Typing](https://devopedia.org/duck-typing) and our `factories` list isn't tied to any specific type. However, in other programming languages that use strong typing, you will see the reference to an abstract base class for the factories.